In [ ]:
import sys
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline
import h5py

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, LocallyConnected2D, Dropout
from keras.models import Model, load_model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras import initializers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, InceptionV3

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import HDF5Matrix, plot_model, print_summary

import numpy.random as rng

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# LOAD DATA

os.chdir('/content/drive/My Drive/MLCarMakeModel')

f=h5py.File('training_set.h5', 'r')
# NOTE! We don't make a in-memory copy of the dataset... So we cannot close the file!

X1=f['X1']   # Tensor with first images in the pair
X2=f['X2']   # Tensor with second images in the pair
y=f['y']     # Tensor with desired output

# Reshape the tensor with the desired output
n=y.shape[0]
y=y[:].reshape((n,1))

In [ ]:
# MODEL SPECIFICATION

input_shape = (150,200,3)

def get_CNN(nFC = 512, dropout_rate = 0.2, weight_decay = 1e-4):

  # Convolutional Neural Network
  model = Sequential()
  model.add(Conv2D(32, (10,10), activation='relu', input_shape=input_shape, strides = 1, kernel_regularizer=l2(weight_decay), name = "CONV1"))
  model.add(MaxPooling2D(name = "POOL1"))
  model.add(Dropout(dropout_rate))
  model.add(Conv2D(16, (9,9), activation='relu', strides = 2, kernel_regularizer=l2(weight_decay), name = "CONV2"))
  model.add(Dropout(dropout_rate))
  model.add(Conv2D(16, (5,5), activation='relu', kernel_regularizer=l2(weight_decay), name = "CONV3"))
  model.add(MaxPooling2D(name = "POOL3"))
  model.add(Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(weight_decay), name = "CONV4"))
  model.add(Flatten())
  model.add(Dense(nFC, activation='relu', kernel_regularizer=l2(weight_decay), name = "FC"))

  return model

def add_hidden_top(pretrained_model, nFC_last, nFC_first = 0, weight_decay = 1e-4):
  layer = 1
  
  model = Sequential()
  model.add(pretrained_model)
  
  #model.add(Dropout(0.5))
  #model.add(MaxPooling2D())
  model.add(Flatten())

  if nFC_first>0:    
    model.add(Dense(nFC_first, activation = 'relu', kernel_regularizer=l2(weight_decay), name = "FC"+str(layer)))
    layer +=1
  
  model.add(Dense(nFC_last, activation = 'relu', kernel_regularizer=l2(weight_decay), name = "FC"+str(layer)))

  return model

def make_siamese(model):

  # Define the tensors for the two input images
  left_input = Input(input_shape, name = "x1")
  right_input = Input(input_shape, name = "x2")

  # Generate the encodings (feature vectors) for the two images
  encoded_l = model(left_input)
  encoded_r = model(right_input)

  # Add a customized layer to compute the absolute difference between the encodings
  L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]), name = "Distance")
  L1_distance = L1_layer([encoded_l, encoded_r])

  # Add a dense layer with a sigmoid to generate the similarity score
  # weight initialised to negative as longer distance should cause predictions closer to zero
  prediction = Dense(1,activation='sigmoid', name = "Score")(L1_distance)

  # Connect the inputs with the outputs
  net = Model(inputs=[left_input,right_input],outputs=prediction)
  return net

In [ ]:
#TRAINING

# create batch, half of class 1, half of class 0
# return numpy array from dataset of size equal to batch_size 

def get_batch(indices):
  indices = np.sort(indices)
  X1set = X1[indices,:,:,:].astype('float32')/255.0
  X2set = X2[indices,:,:,:].astype('float32')/255.0
  yset = y[indices]
  return [X1set, X2set], yset
# May add image augmentation on batch

n = 40000 # rounded down total number of samples
n1 = 20100 # ...with 20100 in class 1

# epochs: number of runs through entire training set
# validation_split: share of n that is part of hold-out validation set
# p: number of batches from training set, containing examples picked randomly each epoch, to be used for stratified cross validation
# batch_size: number of examples in each batch, with half in each class
# returns tuple of validation performance and time duration
def train(model, epochs, validation_split = 0.1, p = 2, batch_size = 64):
  m = int(n*(1-validation_split)) # number of examples in training set
  m_valid = int(n*validation_split) # number of examples in hold-out validation set

  B = int(m//batch_size) # number of batches in training set
  B_valid = int(m_valid//batch_size)
  
  tic = time.time()

  for e in range(epochs):
    print("\nEpoch: ", e+1, r'/', epochs)
    # one batch contains an equal number of examples picked randomly from each class (or strata)
    ind1 = np.random.permutation(m//2) # indices for class 1
    ind0 = np.random.permutation(m//2) + n1 #indices for class 0
    batch_indices = [np.concatenate((ind0[b*batch_size//2: (b + 1)*batch_size//2], ind1[b*batch_size//2: (b + 1)*batch_size//2])) for b in range(B)] # B size list of arrays
    
    steps = 0
    print("[", end='')
    
    prev_train_accuracy = 0
    for indices in batch_indices[:B-p]:
      pairs, targets = get_batch(indices)
      train_loss, train_accuracy = model.train_on_batch(pairs, targets)
      
      if train_accuracy==prev_train_accuracy:
        print("=", end = '')
      elif train_accuracy>prev_train_accuracy:
        print("+", end = '') # + for improving
      else: 
        print("-", end = '') # - for worsening
      steps +=1
      if steps % 100 ==0:
        print('\n')
      prev_train_accuracy = train_accuracy
  
    print("] steps = ", steps, "B-p =", B-p)
    print(" loss:    ", train_loss, " \t-    acc:", train_accuracy)

    # Cross-validation
    for indices in batch_indices[B-p:B]:
      pairs, targets = get_batch(indices)
      cv_loss, cv_acc = model.test_on_batch(pairs, targets)
  
    print(" cv_loss: ", cv_loss, " \t- cv_acc: ", cv_acc)

  ind1 = m//2 + np.random.permutation(m_valid//2) + n1
  ind0 = m//2 + np.random.permutation(m_valid//2)
  batch_indices = [np.concatenate((ind0[b*batch_size//2: (b + 1)*batch_size//2], ind1[b*batch_size//2: (b + 1)*batch_size//2])) for b in range(B_valid)]

  for indices in batch_indices: 
    pairs, target = get_batch(indices)
    val_loss, val_acc = net.test_on_batch(pairs, target)

  print("Evaluation on hold-out validation set")
  print("val_loss = ", val_loss, " - val_acc: ", val_acc)     

  toc = time.time()
  elapsed = int(toc - tic)
  print(elapsed//60, " min ", elapsed%60 , " s ")

  return (val_loss, val_acc, elapsed)


# Check if evaluate on validation set can be used instead - NO crash!                


In [ ]:
# SEARCH FOR HYPERPARAMETERS

# eta = [0.0001, 0.00001, 0.00006 , 0.00001]
# optimizer = [Adam(lr=0.00006), SGD(lr=0.00006, momentum = 0.5)] Adam is well recognised for better training performance at least for the early stages of training

hyperparameters = {'128-0': (128,0), '256-0': (256,0), '128-1024': (128,1024), '128-4096': (128,4096), '256 - 128': (256,128)}
valid_performance = Dict()

vgg = VGG16(include_top=False, weights='imagenet', input_shape=input_shape, pooling = 'true')
vgg.trainable = False

inception = InceptionV3(include_top = False, weights = 'imagenet', input_shape = input_shape, pooling = 'true')
inception.trainable = False

models = [vgg, inception]
modelnames = ["vgg", "inception"]

for i in range(2):
  print(modelnames[i])
  for key in hyperparameters:
    print(key)
    h1, h2 = hyperparameters[key] 
    net = make_siamese(add_hidden_top(model[i], h1, h2))
    net.compile(loss="binary_crossentropy",optimizer=Adam(lr=0.00006), metrics =['binary_accuracy'])
    net.summary()
    valid_performance[key] = train(net, 2)
    net.save_weights(key+'_weights.h5')














1small


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x1 (InputLayer)                 (None, 150, 200, 3)  0                                            
__________________________________________________________________________________________________
x2 (InputLayer)                 (None, 150, 200, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 128)          16287680    x1[0][0]                         
                                                                 x2[0][0]                         
___________________________________________________________________________

In [ ]:
net.predict([X1[0:5].astype('float32')/255.0, X2[0:5].astype('float32')/255.0])

array([[0.58568776],
       [0.58568776],
       [0.58568776],
       [0.58568776],
       [0.58568776]], dtype=float32)

In [ ]:
net.predict([X1[20500:20505].astype('float32')/255.0, X2[20500:20505].astype('float32')/255.0])

NameError: ignored

In [ ]:
# Save weights


model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')


#Assuming you have code for instantiating your model, you can then load the weights you saved into a model with the same architecture:

model.load_weights('my_model_weights.h5')